# Working with HDF5 files

After a long afternoon, I managed to extract some Advanced Microwave Sounding Unit (AMSU) data from EUMETSAT and saved them in the HDF5 format. For details about this and further information about the satellite and sensor, please see the notebook the-eumetsat-saga.ipynb.

You can obtain the 3 example files from Sciebo: https://fz-juelich.sciebo.de/s/A6wUfYMJBn1EpvD . As I didn't find documentation about a sciebo API (although there is a pysciebo package), you will have to download the files manually.

In [ ]:
import os
import xarray
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [ ]:
# make sure, the three files have been downloaded
files = ["AMSAL1_20240430T221919Z_20240501T000119Z_epct_25dad12a_F.h5",
         "AMSAL1_20240501T000119Z_20240501T014319Z_epct_2db170ca_F.h5",
         "AMSAL1_20240501T014319Z_20240501T032823Z_epct_e379cfef_F.h5" ]
for file in files:
    if not(os.path.isfile(file) and os.access(file, os.R_OK)):
        print(f"File {file} is missing. Please download from https://fz-juelich.sciebo.de/s/A6wUfYMJBn1EpvD .")

In [ ]:
# open the files in xarray
datasets = []
for file in files:
    ds = xarray.open_dataset(file)
    print(ds)
    datasets.append(ds)

In [ ]:
print(datasets[0]["lon"].values)

In [ ]:
print(datasets[0]["lat"].values)

In [ ]:
# plot coordinates on a map as points
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
colors = ['r', 'g', 'b']
colors = ['#91AEEC', '#4D81EF', '#0B55F1']
for ds, c in zip(datasets, colors):
    ax.plot(ds.lon, ds.lat, marker='o', color=c, markersize=1, linestyle='None')


In [ ]:
# try to concatenate the three datasets into one
#ds_all = datasets[0].copy(deep=True)
ds_all = xarray.concat(datasets, dim='x')

In [ ]:
print(ds_all)

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.scatter(ds_all.lon, ds_all.lat, marker='o', c=ds_all.channel_3, s=1., cmap='plasma')
plt.show()

In [ ]:
# save data as HDF5 file (note: this is done via to_netcdf with the HDF5 engine
ds_all.to_netcdf("combined_data.h5", engine="h5netcdf")
